<a href="https://colab.research.google.com/github/nrohrbach/OGD_Monitoring_BAFU/blob/main/OGD_Monitoring_BAFU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install pandas

In [65]:
#pip install matplotlib

## Libraries importieren

In [2]:
import requests
import json
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline

## Alle Packages des BAFU abfragen

In [130]:
# Abfrage aller CKAN Packages des BAFU
Packages = "https://ckan.opendata.swiss/api/3/action/organization_show?id=bundesamt-fur-umwelt-bafu&include_datasets=True"
Packages = requests.get(Packages).json()

In [131]:
# Alle Bezeichungen und Publisher extrahieren
Title = [s['title_for_slug'] for s in Packages['result']['packages']]
Maintainer = [s['maintainer'] for s in Packages['result']['packages']]
Email = [s['maintainer_email'] for s in Packages['result']['packages']]
LastModified = [s['modified'] for s in Packages['result']['packages']]

In [132]:
# Dataframe für Packages erstellen
dict = {'Publisher': Maintainer, 'Mail': Email, 'Package': Title, 'LastModified': LastModified}
dfPackages = pd.DataFrame(dict)

In [134]:
# Mit folgender BaseURL können Details aller Packages abgerufen werden
DatasetBaseURL = 'https://ckan.opendata.swiss/api/3/action/package_show?id='

In [135]:
# Alle Package-Detailsabfragen
Datasets = []

for package in Title:
    try:
        DataSet = requests.get(DatasetBaseURL + package).json()
        Maintainer = DataSet['result']['maintainer']
        Email = DataSet['result']['maintainer_email']
        LastModified = DataSet['result']['modified']
        Issued = DataSet['result']['issued']
        License = DataSet['result']['resources'][0]['license']
        Datasets.append({'Package': package, 'Publisher': Maintainer, 'Mail': Email, 'LastModified':LastModified,'Issued':Issued,'License':License})

    except:
        Datasets.append({'Package': package, 'Publisher': 'Unbekannt', 'Mail': 'Unbekannt','LastModified':'','Issued':'','License':''})


In [136]:
# Package Details als Dataframe speichern
dfPackages = pd.DataFrame(Datasets)

In [137]:
# Falls LastModified leer ist, wird der Wert aus Issued übernommen.
dfPackages['LastModified'] = dfPackages.apply(lambda row: row['Issued'] if pd.isnull(row['LastModified']) or row['LastModified'] == '' else row['LastModified'], axis=1)

In [138]:
# Lizenzstring kürzen
dfPackages['License'] = dfPackages['License'].str.split('#').str[1]

In [140]:
# Alle Datasets aller Packages abfragen
Datasets = []

for package in Title:
    try:
        DataSet = requests.get(DatasetBaseURL + package).json()
        # Alle Bezeichungen und Publisher extrahieren
        Format = [s['format'] for s in DataSet['result']['resources']]
        PackageId = [s['package_id'] for s in DataSet['result']['resources']]
        Display_Name = [s['display_name']['de'] for s in DataSet['result']['resources']]
        Datasets.append({'Package': package, 'Format': Format,'Display_Name':Display_Name})

    except:
        Datasets.append({'Package': package, 'Format': 'Unbekannt','Display_Name':''})

In [141]:
# Dataframe der Datasets erstellen und mit dem Dataframe der Packages mergen
dfDatasets = pd.DataFrame(Datasets)
dfDatasets = dfDatasets.explode('Format', ignore_index=False)

dfDatasets = pd.merge(dfDatasets, dfPackages, how='left', on=['Package'])
dfDatasets['Date'] = datetime.today().strftime("%Y-%m-%d")

In [196]:
# Dataframe nur mit Geodaten für STAC Monitoring

# Create a new column 'STAC_Attribut'
dfDatasets['STAC'] = 'nein'

# Update 'STAC_Attribut' to 'ja' where the 'STAC' column contains 'STAC'
dfDatasets.loc[dfDatasets['Display_Name'].astype(str).str.contains('STAC'), 'STAC'] = 'ja'

# Filter the DataFrame to include only entries where 'Display_Name' contains "map.geo.admin.ch"
dfGeodaten = dfDatasets[dfDatasets['Display_Name'].astype(str).str.contains('map.geo.admin.ch')]

dfGeodaten = dfGeodaten[dfGeodaten['Format'] == 'SERVICE']

## Daten als CSV speichern

In [151]:
# Übersicht Packages speichern
dfPackagesCSV = dfPackages.groupby(['Mail'])['Package'].count().reset_index()
dfPackagesCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfPackagesCSV.to_csv("data/BAFU_OGD_Monitoring_Packages.csv", header=False, index=False, mode='a')

In [152]:
# Übersicht Datasets speichern
dfDatasetsCSV = dfDatasets.groupby(['Mail'])['Package'].count().reset_index()
dfDatasetsCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfDatasetsCSV.to_csv("data/BAFU_OGD_Monitoring_Datasets.csv", header=False, index=False, mode='a')

In [153]:
# Übersicht Formate speichern
dfFormats = dfDatasets.groupby(['Format'])['Mail'].count().reset_index()
dfFormats.replace("","Fehlt",inplace=True)
dfFormats['Date'] = datetime.today().strftime("%Y-%m-%d")
dfFormats.to_csv("data/BAFU_OGD_Monitoring_Formats.csv", header=False, index=False, mode='a')

In [154]:
# Übersicht Last Update speichern
dfPackages['LastModified'] = dfPackages['LastModified'].str.slice(0,7)
dfLastUpdateCSV = dfPackages.groupby(['LastModified'])['Mail'].count().reset_index()
dfLastUpdateCSV.replace("","1999-01",inplace=True) #Fehlende Werte werden 1999 dargestellt.
dfLastUpdateCSV['LastModified'] = pd.to_datetime(dfLastUpdateCSV['LastModified'])
dfLastUpdateCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfLastUpdateCSV.to_csv("data/BAFU_OGD_Monitoring_LastUpdate.csv", header=False, index=False, mode='a')

In [118]:
# Übersicht Lizenz speichern
dfLicenseCSV = dfDatasets.groupby(['License'])['Package'].count().reset_index()
dfLicenseCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfLicenseCSV.to_csv("data/BAFU_OGD_Monitoring_License.csv", header=False, index=False, mode='a')

In [150]:
# Übersicht STAC speichern
dfGeodatenCSV = dfGeodaten.groupby(['Mail','STAC'])['Package'].count().reset_index()
dfGeodatenCSV['Date'] = datetime.today().strftime("%Y-%m-%d")
dfGeodatenCSV.to_csv("data/BAFU_OGD_Monitoring_STAC.csv", header=False, index=False, mode='a')

## Visualisierungen erstellen

In [155]:
#Packages - Daten reinladen
dfPackages = pd.read_csv("data/BAFU_OGD_Monitoring_Packages.csv", parse_dates=['Date'])

ValueError: Missing column provided to 'parse_dates': 'Date'

In [ ]:
# Linechart mit allen BAFU OGD-Publikationen
dfPackagesBAFU = dfPackages.groupby(dfPackages['Date']).sum()
dfPackagesBAFU = dfPackagesBAFU.drop("Mail", axis='columns')

dfPackagesBAFU.plot(figsize=(15,10))
plt.legend(loc='upper left')
plt.title("Anzahl OGD Publikationen")
plt.savefig('plots/PackagesBAFULinechart.png')
plt.close()

In [ ]:
#Packages Barchart
dfPackagesBar = dfPackages.loc[dfPackages['Date']==datetime.today().strftime("%Y-%m-%d")]
dfPackagesBar = dfPackagesBar.sort_values('Package',ascending=False)

fig, ax = plt.subplots()
ax.bar(dfPackagesBar['Mail'], dfPackagesBar['Package'])
ax.set_title('Anzahl OGD Publikationen')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Masten")
plt.rcParams["figure.figsize"] = (10,5)
ax.bar_label(ax.containers[0], label_type='edge')
plt.savefig('plots/PackagesBarchart.png',bbox_inches='tight')
plt.close()

In [ ]:
#Packages - Linechart
dfPackagesLine = dfPackages.pivot(index="Date", columns=['Mail'],values="Package")

dfPackagesLine.plot(figsize=(15,10))
plt.legend(loc='lower left')
plt.title("Anzahl OGD Publikationen")
plt.savefig('plots/PackagesLinechart.png')
plt.close()

In [ ]:
#Datasets - Daten reinladen
dfDatasets = pd.read_csv("data/BAFU_OGD_Monitoring_Datasets.csv", parse_dates=['Date'])

In [ ]:
#Datasets Barchart
dfDatasetsBar = dfDatasets.loc[dfDatasets['Date']==datetime.today().strftime("%Y-%m-%d")]
dfDatasetsBar = dfDatasetsBar.sort_values('Package',ascending=False)

fig, ax = plt.subplots()
ax.bar(dfDatasetsBar['Mail'], dfDatasetsBar['Package'])
ax.set_title('Anzahl OGD Datensätze')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Masten")
plt.rcParams["figure.figsize"] = (10,5)
ax.bar_label(ax.containers[0], label_type='edge')
plt.savefig('plots/DatasetsBarchart.png',bbox_inches='tight')
plt.close()

In [ ]:
#Datsets - Linechart
dfDatasetsLine = dfDatasets.pivot(index="Date", columns=['Mail'],values="Package")

dfDatasetsLine.plot(figsize=(15,10))
plt.legend(loc='lower left')
plt.title("Anzahl OGD Datensätze")
plt.savefig('plots/DatasetsLinechart.png')
plt.close()

In [ ]:
#Formate - Daten reinladen
dfFormats = pd.read_csv("data/BAFU_OGD_Monitoring_Formats.csv", parse_dates=['Date'])

In [ ]:
#Formats Barchart
dfFormatsBar = dfFormats.loc[dfFormats['Date']==datetime.today().strftime("%Y-%m-%d")]
dfFormatsBar = dfFormatsBar.sort_values('Mail',ascending=False)

fig, ax = plt.subplots()
ax.bar(dfFormatsBar['Format'], dfFormatsBar['Mail'])
ax.set_title('Anzahl OGD Datensätze nach Format')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Datensätze")
plt.rcParams["figure.figsize"] = (10,5)
ax.bar_label(ax.containers[0], label_type='edge')
plt.savefig('plots/FormatssBarchart.png',bbox_inches='tight')
plt.close()

In [ ]:
#Formats - Linechart
dfFormatsLine = dfFormats.pivot(index="Date", columns=['Format'],values="Mail")

dfFormatsLine.plot(figsize=(15,10))
plt.legend(loc='lower left')
plt.title("Anzahl OGD Datensätze nach Format")
plt.savefig('plots/FormatssLinechart.png')
plt.close()

In [ ]:
#Last Update - Daten reinladen
dfLastUpdate = pd.read_csv("data/BAFU_OGD_Monitoring_LastUpdate.csv", parse_dates=['Date'])
dfLastUpdate = dfLastUpdate.loc[dfLastUpdate['Date']==datetime.today().strftime("%Y-%m-%d")]
dfLastUpdate['LastModified'] = pd.to_datetime(dfLastUpdate['LastModified'])
dfLastUpdate['Mail']=pd.to_numeric(dfLastUpdate['Mail'])

In [ ]:
#Last Update - Barchart
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"] = (4,2)
plt.bar(x=dfLastUpdate['LastModified'], height=dfLastUpdate['Mail'], width=20)
ax.set_title('Anzahl Publikationen')
plt.xticks(rotation = 90)
plt.ylabel("Anzahl Publikationen")
ax.bar_label(ax.containers[0], label_type='edge')
plt.rcParams["figure.figsize"] = (10,5)
plt.savefig('plots/LastUpdate.png',bbox_inches='tight')
plt.close()

In [164]:
#License - Linechart
dfLicense = pd.read_csv("data/BAFU_OGD_Monitoring_License.csv", parse_dates=['Date'])
dfLicense = dfLicense.pivot(index="Date", columns=['License'],values="Package")
dfLicense.plot(figsize=(15,10))
plt.title("Anzahl OGD Datensätze nach Lizenz")
plt.savefig('plots/LizenzLinechart.png')
plt.close()

In [195]:
#STAC NEIN
dfGeodaten = pd.read_csv("data/BAFU_OGD_Monitoring_STAC.csv", parse_dates=['Date'])
dfGeodaten = dfGeodaten[dfGeodaten['STAC'] == 'nein']
dfGeodaten = dfGeodaten.pivot(index="Date", columns=['Mail'],values="Package")
dfGeodaten.plot(figsize=(15,10))
plt.title("Anzahl Geo-Datensätze welche noch nicht auf STAC verfügbar sind")
plt.savefig('plots/StacLinechart.png')
plt.close()